In [ ]:
#!/Python34/python

In [4]:
import os
import requests
import time

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#### Lookup

The city id's for Turkey is in `noaa_lookup.txt` file

In [5]:
with open('noaa_lookup.txt', 'r') as file:
    num_list = [str(line) for line in file]

In [6]:
YEAR = '2018/'

In [9]:
# construct Turkey's id as a list

id_turkey = []

for idx, line in enumerate(num_list):
    if idx in (0,1):
        continue
    else:
        id_turkey.append(line.split()[0]+line.split()[1]);

### Scraping 

Scraping the hourly weather for each city of Turkey and printing the number of cities remaining as output

In [ ]:
base_url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + YEAR

for ix, id in enumerate(id_turkey):
    file_name = id + '.csv'
    url = base_url + file_name
    print('Number of cities remaining:', len(id_turkey) - (ix+1))
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    r = session.get(url)

    with open(file_name, "wb") as text_file:
        text_file.write(r.content)